In [1]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyLDavis

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install python-Levenshtein

Note: you may need to restart the kernel to use updated packages.


In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
nlp = spacy.load("en_core_web_sm")

In [5]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
#lda_viz = gensimvis.prepare(ldamodel, corpus, dictionary)import matplotlib.pyplot as plt
%matplotlib inline

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecation

In [6]:
import nltk; nltk.download('stopwords')

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/image.py:172: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
[nltk_data] Downloadi

True

In [7]:
from nltk.corpus import stopwords

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
documents = pd.read_excel("CDP.xlsx")

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/openpyxl/compat/numbers.py:41: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  numpy.float,


In [9]:
df = ["Agree with Rajan on a lot of points - especially the vaccine unpreparedness issues.However the notion that our current form of governance has eroded foreign investor confidence is untrue and he knows it well enough. It's literally at an all time high. Another point that seems politically driven - If you are criticising the economic recovery of phase 1 based on the fact that it is all due to Govt exp and not a consumption rise here's 3 points for you: 1. Same trend globally. Every govt is spending to revive the economy. Weren't you advocating a no holds barred currency printing last year yourself? There's no way pvt consumption - that was falling in India even pre covid btw - could have revived last year. 2. This is natural during any economic jolt. What  happened in 2009-2011? Are you claiming Congress govt revived pvt consumption when we saw those double digit growth figures? We all know the subsequent inflationary trends that lasted till 2013. It was all govt spending. 3. Growth is a factor of C+I+G+NX If we open up, we risk more dead people and eventually C will fall either way. Investment is actually growing. FDI is nearly at all time highs. You cannot expect exports to grow exponentially given the global problem. That leaves govt expenditure"]

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
documents = documents[["Comment", "Score", "Driver"]]

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
documents

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Comment,Score,Driver
0,"Yes, I am happy till the time it doesn't cost ...",8.0,Diversity and inclusion
1,Highly satisfied as I did not feel gender part...,0.0,Diversity and inclusion
2,I did not found any partiality due to the back...,10.0,Non-discrimination
3,Because every day I am surrounded by passionat...,10.0,Inclusiveness
4,Almost all of my colleagues rewarded and promo...,1.0,Inclusiveness
...,...,...,...
241,not sure,6.0,Engagement
242,not sure,6.0,Organizational fit
243,yes,8.0,Autonomy
244,Yes,8.0,z. COVID-19


In [12]:
documents = documents.dropna(axis=0)

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
documents.shape

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(229, 3)

In [14]:
documents.head()

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Comment,Score,Driver
0,"Yes, I am happy till the time it doesn't cost ...",8.0,Diversity and inclusion
1,Highly satisfied as I did not feel gender part...,0.0,Diversity and inclusion
2,I did not found any partiality due to the back...,10.0,Non-discrimination
3,Because every day I am surrounded by passionat...,10.0,Inclusiveness
4,Almost all of my colleagues rewarded and promo...,1.0,Inclusiveness


In [15]:
#stop_words
# Convert to list
#data = df.content.values.tolist()
# Remove Emails
#data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
#data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
#data = [re.sub("\'", "", sent) for sent in data]
#pprint(data[:1])

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
documents.head()

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Comment,Score,Driver
0,"Yes, I am happy till the time it doesn't cost ...",8.0,Diversity and inclusion
1,Highly satisfied as I did not feel gender part...,0.0,Diversity and inclusion
2,I did not found any partiality due to the back...,10.0,Non-discrimination
3,Because every day I am surrounded by passionat...,10.0,Inclusiveness
4,Almost all of my colleagues rewarded and promo...,1.0,Inclusiveness


In [18]:
docs = documents["Comment"]
docs

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0      Yes, I am happy till the time it doesn't cost ...
1      Highly satisfied as I did not feel gender part...
2      I did not found any partiality due to the back...
3      Because every day I am surrounded by passionat...
4      Almost all of my colleagues rewarded and promo...
                             ...                        
240                                            Yes doing
241                                             not sure
242                                             not sure
243                                                  yes
244                                                  Yes
Name: Comment, Length: 229, dtype: object

In [19]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', ])

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
docs = list(sent_to_words(docs))

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
print(docs[:1])

[['yes', 'am', 'happy', 'till', 'the', 'time', 'it', 'doesn', 'cost', 'the', 'talent', 'men', 'workforce', 'women', 'shouldn', 'be', 'given', 'preference', 'just', 'because', 'they', 'are', 'women', 'but', 'competency', 'what', 'they', 'have', 'but', 'being', 'human', 'we', 'can', 'stop', 'being', 'biased', 'and', 'fear', 'this', 'biasness']]


/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
bigram = gensim.models.Phrases(docs, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[docs], threshold=100)  

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
print(trigram_mod[bigram_mod[docs[0]]])

['yes', 'am', 'happy', 'till', 'the', 'time', 'it', 'doesn', 'cost', 'the', 'talent', 'men', 'workforce', 'women', 'shouldn', 'be', 'given', 'preference', 'just', 'because', 'they', 'are', 'women', 'but', 'competency', 'what', 'they', 'have', 'but', 'being', 'human', 'we', 'can', 'stop', 'being', 'biased', 'and', 'fear', 'this', 'biasness']


/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
# Remove Stop Words
docs_nostops = remove_stopwords(docs)

# Form Bigrams
docs_bigrams = make_bigrams(docs_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm")

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(docs_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['happy', 'time', 'cost', 'talent', 'man', 'workforce', 'woman', 'give', 'preference', 'woman', 'competency', 'human', 'stop', 'bias', 'fear', 'biasness']]


In [27]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1)]]


/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[('bias', 1),
  ('biasness', 1),
  ('competency', 1),
  ('cost', 1),
  ('fear', 1),
  ('give', 1),
  ('happy', 1),
  ('human', 1),
  ('man', 1),
  ('preference', 1),
  ('stop', 1),
  ('talent', 1),
  ('time', 1),
  ('woman', 2),
  ('workforce', 1)]]

In [29]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.096*"get" + 0.056*"month" + 0.053*"offer" + 0.052*"also" + 0.048*"skill" '
  '+ 0.035*"really" + 0.032*"due" + 0.020*"stay" + 0.018*"experience" + '
  '0.016*"increment"'),
 (1,
  '0.223*"agree" + 0.048*"consider" + 0.031*"individual" + 0.029*"give" + '
  '0.014*"mo" + 0.014*"resource" + 0.014*"respect" + 0.014*"concern" + '
  '0.001*"suggestion" + 0.001*"improve"'),
 (2,
  '0.191*"team" + 0.068*"definitely" + 0.029*"right" + 0.015*"suppotive" + '
  '0.001*"unethical" + 0.001*"report" + 0.001*"behavior" + 0.001*"physical" + '
  '0.001*"impact" + 0.001*"location"'),
 (3,
  '0.223*"engagement" + 0.165*"current" + 0.101*"thing" + 0.055*"way" + '
  '0.042*"contribution" + 0.021*"deliver" + 0.013*"environment" + '
  '0.013*"never" + 0.011*"reward" + 0.011*"ethical"'),
 (4,
  '0.132*"manager" + 0.123*"previous" + 0.118*"learn" + 0.077*"support" + '
  '0.065*"always" + 0.027*"change" + 0.019*"positive" + 0.014*"monthly" + '
  '0.014*"require" + 0.014*"answer"'),
 (5,
  '0.326*"work"

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -11.10505175644605

Coherence Score:  0.4612978322191615


In [32]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis
pyLDAvis.save_html(vis, 'CDP.html')

/Users/tadatta/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
